In [1]:
%%html
<style>
.h1_cell, .just_text {
    box-sizing: border-box;
    padding-top:5px;
    padding-bottom:5px;
    font-family: "Times New Roman", Georgia, Serif;
    font-size: 125%;
    line-height: 22px; /* 5px +12px + 5px */
    text-indent: 25px;
    background-color: #fbfbea;
    padding: 10px;
}
.code_block {
    box-sizing: border-box;
    padding-top:5px;
    padding-bottom:5px;
    font-size: 75%;
    line-height: 22px; /* 5px +12px + 5px */
    #text-indent: 25px;
    #background-color: #fbfbea;
    padding: 5px;
}

hr { 
    display: block;
    margin-top: 0.5em;
    margin-bottom: 0.5em;
    margin-left: auto;
    margin-right: auto;
    border-style: inset;
    border-width: 2px;
}
</style>

<h2>
<center>
Can we Parallelize KNN?
</center>
</h2>

In [2]:
import os
import sys
import subprocess

In [3]:
os.environ['SPARK_HOME'] = os.environ['HOME'] + '/spark'
os.environ['PATH'] += ':' + os.environ['SPARK_HOME'] + '/bin'
sys.path.append(os.environ['SPARK_HOME'] + '/python')
sys.path.append(os.environ['SPARK_HOME'] + '/python/lib/py4j-0.10.6-src.zip')

In [4]:
# run start-all.sh
subprocess.call(os.environ['SPARK_HOME'] + "/sbin/start-all.sh", env=os.environ)

0

In [5]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
spark = SparkSession(SparkContext(master='spark://blue0:7077'))
spark

<div class=h1_cell>
<p>
Sometimes its better think about these problems from the bottom up. Is there a row-wise operation that can be run on the dataset in parallel?
<p>
Yes. Given a row, we can calculate the distance from every other row in the distributed dataset.
<p>
Lets import our dataset using pandas and use our NLP code from pyspark_lsa.ipynb to process the text.
</div>

In [6]:
if(True):  # if on raspberry pi, change if otherwise
    subprocess.call('sudo apt-get install python-pandas'.split(' '))
else:
    subprocess.call('pip install pandas'.split(' '))

import pandas as pd

gothic_table = pd.read_csv('https://bit.ly/2HVSx3X', encoding='utf-8')
gothic_table.head(5)

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [7]:
from nlp import get_bag_and_tokenize
from lnalg import occurence_matrix

bag, sentences = get_bag_and_tokenize(gothic_table.head(10), 'text')
omatrix = occurence_matrix(bag, sentences)
omatrix.head(5)

[nltk_data] Downloading package stopwords to /home/pi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,code,gold,incessantly,desk,fantastic,years,attempts,terrace,mentioned,yet,...,without,looked,greek,cannot,greatest,steals,left,avoid,fact,usual
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
3,0,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [8]:
import numpy as np
from random import random
from pyspark.sql.functions import udf, col
from pyspark.sql.types import FloatType
from pyspark.ml.feature import VectorAssembler, VectorDisassembler

In [52]:
randv = np.random.rand(len(omatrix.columns))
dist = udf(lambda x: np.linalg.norm(x-randv), FloatType())

df = spark.createDataFrame(omatrix.values.tolist(), schema=list(omatrix.columns))
vdf = VectorAssembler(inputCols=list(omatrix.columns), outputCol='features').transform(df)
vdf = vdf.withColumn('distance', dist(vdf.features))
def a(x): lambda x: 
#vdf = vdf.drop('features')
#vdf.select('distance').rdd.collect()

TypeError: 'Column' object is not callable